# Convolutional Neural Networks

![LeNet5](https://miro.medium.com/max/4308/1*1TI1aGBZ4dybR6__DI9dzA.png)

Convolutional Neural Networks are particularly well suited for image classification. In https://www.tensorflow.org/tutorials/images/cnn and https://keras.io/examples/vision/mnist_convnet some well performing examples are given. First, modify the network to include a
single convolutional layer, a single pooling layer, and a single fully connected layer only.  
Then, incrementally add more complexity and regularization, particularly:
- consider the network architecture of LeNet5 (shown in the figure) and get inspired by AlexNet.
  (As AlexNet was designed for much larger input images (227x227) it cannot be applied 1:1 to MNIST images (28x28)).
- try to further improve the test error
  (Remember to only use your validation set to tweak your hyper parameters!)

Record training and test accuracies with TensorBoard.  
How many parameters have your networks, also compared to a standard 128-64-64-10 MLP network from last sheet?  
How long does it take to train them?

In [33]:
import tensorflow as tf
import keras.backend as K
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorboard as tb
import numpy as np
from datetime import datetime
import pandas as pd

In [19]:
# Cifar10 dataset

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0

In [27]:
# MNNIST dataset

num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

In [ ]:
# Lets start with the single convolutional layer, a single pooling layer, and a single fully connected layer only.

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

log_dir = "./logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.summary()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, validation_split=0.1)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)



In [42]:
# LeNet5 model

model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

model.summary()
# Tensorboard training and test accuracy

log_dir = "./logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=2)

test_loss, test_acc = model.evaluate(test_images,  test_labels)

print(test_acc)

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 15, 15, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_41 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_29 (MaxPooli  (None, 6, 6, 64)          0         
 ng2D)                                                           
                                                                 
 conv2d_42 (Conv2D)          (None, 4, 4, 64)          36928     
                                                                 
 flatten_17 (Flatten)        (None, 1024)            

In [41]:
# Further improvements --> add more layers and batch normalization

model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=2)

test_loss, test_acc = model.evaluate(test_images,  test_labels)

print(test_acc)

Epoch 1/2
1563/1563 [==============================] - 33s 21ms/step - loss: 1.2637 - accuracy: 0.5511
Epoch 2/2
313/313 [==============================] - 2s 7ms/step - loss: 0.9454 - accuracy: 0.6796
0.6796000003814697


In [ ]:
# call tensorboard
%tensorboard --logdir ./logs

By increasing the number of layers and adding normalization we increased the overall accuracy by 5% (from 63 to 68) but the runtime increased by almost 3x. 

The first simple model has 72906 parameters and the second has 66570 parameters which is a little more compared to the models of the last sheet.